In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from agents import input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

# Load environment variables
load_dotenv(override=True)

# Get API keys
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

# Print status of keys
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

# Base URLs
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

# Set up model clients
ollama_client = AsyncOpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

# Define models
qwen_model = OpenAIChatCompletionsModel(model="qwen:7b", openai_client=ollama_client)
llama3_model = OpenAIChatCompletionsModel(model="llama3:latest", openai_client=ollama_client)
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-r1:latest", openai_client=ollama_client)
phi_model = OpenAIChatCompletionsModel(model="phi4:latest", openai_client=ollama_client)
qroq_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

# Instructions
instructions1 = (
    "You are a sales agent working for ComplAI, a company that provides a SaaS tool "
    "for ensuring SOC2 compliance and preparing for audits, powered by AI. "
    "You write professional, serious cold emails."
)

instructions2 = (
    "You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool "
    "for ensuring SOC2 compliance and preparing for audits, powered by AI. "
    "You write witty, engaging cold emails that are likely to get a response."
)

instructions3 = (
    "You are a busy sales agent working for ComplAI, a company that provides a SaaS tool "
    "for ensuring SOC2 compliance and preparing for audits, powered by AI. "
    "You write concise, to the point cold emails."
)

# Agents
sales_agent1 = Agent(name="Qwen Sales Agent", instructions=instructions1, model=qwen_model)
sales_agent2 = Agent(name="Llama3 Sales Agent", instructions=instructions2, model=llama3_model)
sales_agent3 = Agent(name="DeepSeek Sales Agent", instructions=instructions3, model=deepseek_model)

description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

# Email sending tool
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body to all sales prospects."""
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email("mano@ieiworld.com")
    to_email = To("nmanomathew066@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success", "response_code": response.status_code}

# Tools for formatting email
subject_instructions = (
    "You can write a subject for a cold sales email. You are given a message and you need to "
    "write a subject for an email that is likely to get a response."
)

html_instructions = (
    "You can convert a text email body to an HTML email body. You are given a text email body which "
    "might have some markdown and you need to convert it to an HTML email body with simple, clear, compelling layout and design."
)

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=deepseek_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=deepseek_model)
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert text email body to HTML")

# Email manager agent
email_tools = [subject_tool, html_tool, send_html_email]
emailer_instructions = (
    "You are an email formatter and sender. You receive the body of an email to be sent. "
    "You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. "
    "Finally, you use the send_html_email tool to send the email with the subject and HTML body."
)

emailer_agent = Agent(
    name="Email Manager",
    instructions=emailer_instructions,
    tools=email_tools,
    model=qroq_model,
    handoff_description="Convert an email to HTML and send it"
)

# Final decision-making sales manager agent
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

sales_manager_instructions = (
    "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. "
    "You never generate sales emails yourself; you always use the tools. "
    "You try all 3 sales agent tools at least once before choosing the best one. "
    "You can use the tools only two times if you're not satisfied with the results from the first try and then you should select the best one. "
    "You select the single best email using your own judgement of which email will be most effective. "
    "After picking the email, you handoff to the Email Manager agent to format and send the email."
)

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=qroq_model
)

# Run the pipeline
message = "Send out a cold sales email addressed to Dear CEO from Alice"

import asyncio

result = await Runner.run(sales_manager, message)
print(result)


OpenAI API Key exists and begins sk-proj-
Google API Key not set (and this is optional)
DeepSeek API Key not set (and this is optional)
Groq API Key exists and begins gsk_


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jgzxaa9qfay90r0tqtpbxfy9` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98879, Requested 5894. Please try again in 1h8m43.532999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}